In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import pandas as pd
import numpy as np
import time
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

MODEL_PATH = ''


train_df = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv',index_col='ID_code')
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv',index_col='ID_code')

synthetic_indices = np.load('../input/synthetissantandersamples/synthetic_samples_indexes.npy')
mask=np.full(len(test_df),True,dtype=bool)
mask[synthetic_indices]=False
test_df_nonsynthetic = test_df.iloc[mask].reset_index(drop=True).copy()


y = train_df.pop('target')
target = y

tr_te = pd.concat([train_df,test_df])

num_cols = [c for c in train_df.columns]

for f in tqdm(num_cols):
    tr_te[f+'_counts'] = tr_te[f].map(pd.concat([train_df[f], test_df_nonsynthetic[f]], axis=0).value_counts().to_dict(), na_action='ignore')
    tr_te[f+'_counts'] = tr_te[f+'_counts'].fillna(1)


count_cols = [f+'_counts' for f in num_cols]


from scipy.special import erfinv
from scipy.stats import rankdata

def rankgauss(x):
    r = (rankdata(x) - 1) / len(x)  # to [0,1]
    r = 2 * r - 1  # to [-1,1]
    r = np.clip(r, -0.99, 0.99)
    r2 = erfinv(r)
    return r2